# Analyse french stocks

In this tutorial, we will use the basic technics to analyse all french stock and try to find out which stock is worth for investment.

The objectives:
1. Find the top 20 stock with the best dividendYield of the last 5 years
2. Find the top 20 stock which are under evaluated ()
3. Find the top 20 stock which lost values in percentage.

## 1. Get all french stock symbols (ticker)

We can download the list of the stock symbols from this [site](https://www.abcbourse.com/download/libelles)



In [5]:
import pandas as pd
import yfinance as yf
import datetime

In [2]:
filePath= "../data/fr_stocks.csv"

stockSymbolDf = pd.read_csv(filePath, sep=";")
stockSymbolDf.head(5)

,ISIN,nom,ticker
0,FR0000031122,Air France - KLM,AF
1,FR0000031577,Virbac,VIRP
2,FR0000031775,Vicat,VCT
3,FR0000032278,Latecoere,LAT
4,FR0000032526,Guerbet,GBT


In [11]:
# we have 216 stock in our list
stockSymbolDf.shape

(216, 3)

We can notice that the ticker is not valid for yf, We need to add `.PA` to complete. For example, we need to convert `AF` to `AF.PA`

In [8]:
bad = yf.Ticker("AF")
print(bad.info)

{'exchange': 'YHD', 'quoteType': 'MUTUALFUND', 'symbol': 'AF', 'underlyingSymbol': 'AF', 'shortName': '411094', 'timeZoneFullName': 'America/New_York', 'timeZoneShortName': 'EDT', 'uuid': '06fa567b-72d7-3b7e-bea7-46973fbbd1d5', 'gmtOffSetMilliseconds': -14400000, 'maxAge': 86400, 'trailingPegRatio': None}


In [9]:
good=yf.Ticker("AF.PA")
print(good.info)

{'address1': '7 rue du Cirque', 'city': 'Paris', 'zip': '75008', 'country': 'France', 'phone': '33 1 43 17 20 20', 'website': 'https://www.airfranceklm.com', 'industry': 'Airlines', 'industryDisp': 'Airlines', 'sector': 'Industrials', 'sectorDisp': 'Industrials', 'longBusinessSummary': 'Air France-KLM SA, together with its subsidiaries, provides passenger and cargo transportation services and aeronautical maintenance in Metropolitan France, Benelux, rest of Europe, and internationally. The company operates through three segments: Airframe maintenance, Engine Maintenance, and Component Support. It offers airframe maintenance, electronic, mechanical, pneumatic, hydraulic, and other services. The company was founded in 1919 and is headquartered in Paris, France.', 'fullTimeEmployees': 74580, 'companyOfficers': [{'maxAge': 1, 'name': 'Mr. Benjamin M. Smith', 'age': 51, 'title': 'CEO & Director', 'yearBorn': 1971, 'fiscalYear': 2022, 'totalPay': 2235149, 'exercisedValue': 0, 'unexercisedVal

In [10]:
# convert
postfix=".PA"
stockSymbolDf['ticker'] = stockSymbolDf['ticker']+postfix
stockSymbolDf.head(5)

,ISIN,nom,ticker
0,FR0000031122,Air France - KLM,AF.PA
1,FR0000031577,Virbac,VIRP.PA
2,FR0000031775,Vicat,VCT.PA
3,FR0000032278,Latecoere,LAT.PA
4,FR0000032526,Guerbet,GBT.PA


## 2. Enrich the stock data frame

We need more information to judge a stock.

### 2.1 Which sector the enterprise works on ?

In [13]:
tickerList = stockSymbolDf['ticker'].tolist()

tickerDictList = []
for ticker in tickerList:
    tickerInfo = yf.Ticker(ticker).info
    tickerDictList.append(tickerInfo)


tickerDf = pd.DataFrame(tickerDictList)
tickerDf.head(5)

,address1,city,zip,country,phone,website,industry,industryDisp,sector,sectorDisp,...,financialCurrency,trailingPegRatio,address2,dividendRate,dividendYield,priceToBook,debtToEquity,returnOnEquity,fiveYearAvgDividendYield,fax
0,7 rue du Cirque,Paris,75008,France,33 1 43 17 20 20,https://www.airfranceklm.com,Airlines,Airlines,Industrials,Industrials,...,EUR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,13e rue LID - BP 27,Carros,06511,France,33 4 92 08 71 00,https://corporate.virbac.com,Drug Manufacturers—General,Drug Manufacturers—General,Healthcare,Healthcare,...,EUR,NaN,Cedex,1.32,0.0048,2.635232,11.545,0.15512,NaN,NaN
2,4 Rue Aristide Berges,L'Isle-d'Abeau,38080,France,33 4 74 27 59 00,https://www.vicat.fr,Building Materials,Building Materials,Basic Materials,Basic Materials,...,EUR,NaN,Les Trois Vallons,1.65,0.0518,0.546824,73.292,0.06851,4.69,NaN
3,"135, rue de Periole",Toulouse,31079,France,33 5 61 58 77 00,https://www.latecoere.aero,Aerospace & Defense,Aerospace & Defense,Industrials,Industrials,...,EUR,NaN,BP 25211 Cedex 5,NaN,NaN,2.962500,1839.217,-1.00543,NaN,NaN
4,"15, rue des Vanesses",Villepinte,93420,France,33 1 45 91 50 00,https://www.guerbet.com,Medical Devices,Medical Devices,Healthcare,Healthcare,...,EUR,NaN,Zone Paris Nord II,0.50,0.0230,0.708182,82.209,-0.10480,2.60,33 1 45 91 51 99


In [14]:
# set the symbol colum as the index column.
# The inplace=True parameter updates the DataFrame in place, modifying it without needing to assign the result back to the variable.
tickerDf.set_index("symbol",inplace=True)
tickerDf.head(5)

,address1,city,zip,country,phone,website,industry,industryDisp,sector,sectorDisp,...,financialCurrency,trailingPegRatio,address2,dividendRate,dividendYield,priceToBook,debtToEquity,returnOnEquity,fiveYearAvgDividendYield,fax
symbol,,,,,,,,,,,,,,,,,,,,,
AF.PA,7 rue du Cirque,Paris,75008,France,33 1 43 17 20 20,https://www.airfranceklm.com,Airlines,Airlines,Industrials,Industrials,...,EUR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
VIRP.PA,13e rue LID - BP 27,Carros,06511,France,33 4 92 08 71 00,https://corporate.virbac.com,Drug Manufacturers—General,Drug Manufacturers—General,Healthcare,Healthcare,...,EUR,NaN,Cedex,1.32,0.0048,2.635232,11.545,0.15512,NaN,NaN
VCT.PA,4 Rue Aristide Berges,L'Isle-d'Abeau,38080,France,33 4 74 27 59 00,https://www.vicat.fr,Building Materials,Building Materials,Basic Materials,Basic Materials,...,EUR,NaN,Les Trois Vallons,1.65,0.0518,0.546824,73.292,0.06851,4.69,NaN
LAT.PA,"135, rue de Periole",Toulouse,31079,France,33 5 61 58 77 00,https://www.latecoere.aero,Aerospace & Defense,Aerospace & Defense,Industrials,Industrials,...,EUR,NaN,BP 25211 Cedex 5,NaN,NaN,2.962500,1839.217,-1.00543,NaN,NaN
GBT.PA,"15, rue des Vanesses",Villepinte,93420,France,33 1 45 91 50 00,https://www.guerbet.com,Medical Devices,Medical Devices,Healthcare,Healthcare,...,EUR,NaN,Zone Paris Nord II,0.50,0.0230,0.708182,82.209,-0.10480,2.60,33 1 45 91 51 99


In [16]:
outPath= "../data/fr_stock_info.csv"

In [17]:
tickerDf.to_csv(outPath,index=True)

## 3. Get the top 20 stock of dividend yield

In [43]:
fundamentals = ["shortName","longBusinessSummary","dividendYield", "marketCap", "beta", "forwardPE","currentPrice","targetHighPrice","targetLowPrice"]
sortByDiv=tickerDf[fundamentals].sort_values(by="dividendYield",ascending=False)
sortByDiv.head(20)

,shortName,longBusinessSummary,dividendYield,marketCap,beta,forwardPE,currentPrice,targetHighPrice,targetLowPrice
symbol,,,,,,,,,
NXI.PA,NEXITY,Nexity SA operates as a real estate company in...,0.1809,7.596621e+08,1.236563,6.408450,13.650,28.9,15.0
ICAD.PA,ICADE,As a commercial property investor (portfolio w...,0.1310,2.495086e+09,1.179031,7.377575,32.240,40.0,40.0
ALD.PA,ALD,ALD S.A. provides service leasing and vehicle ...,0.1215,7.408224e+09,0.953822,5.213018,8.810,20.1,15.0
ALTA.PA,ALTAREA,Altarea is the French leader in low-carbon urb...,0.1199,1.728011e+09,1.083270,14.850087,84.200,103.0,101.0
MERY.PA,MERCIALYS,Mercialys is one of France's leading real esta...,0.1191,7.414276e+08,1.571158,8.777778,7.900,12.0,7.5
COFA.PA,COFACE,"COFACE SA, through its subsidiaries, provides ...",0.1177,1.930410e+09,0.880393,8.736486,12.930,17.2,13.0
ENGI.PA,ENGIE,"ENGIE SA engages in the power, natural gas, an...",0.0965,3.603044e+10,0.997728,8.653488,14.884,21.2,14.2
KOF.PA,KAUFMAN ET BROAD,Kaufman & Broad S.A. operates as a property de...,0.0958,5.229205e+08,1.055261,9.132841,24.750,35.0,24.5
COV.PA,COVIVIO,"Thanks to its partnering history, its real est...",0.0925,4.003320e+09,1.460635,9.638554,40.000,62.0,44.0


The above dividendYield is the current year. In our case, it's 2023. If we want to get the historical value, we need to calculate by ourselves. The formula is pretty simple : `divYield = currentDiv / currentPrice`.

In [22]:
# we need a function which calculate the actual dividendYield
def getDividendYield(stockTicker:str, currentYear:int=None):
    """
    This function returns the dividendYield of a given year. If year is not provided, use the current year
    :param stockTicker:
    :type stockTicker:
    :param currentYear:
    :type currentYear:
    :return:
    :rtype:
    """
    stock = yf.Ticker(stockTicker)
    if not currentYear:
        currentYear=datetime.datetime.now().year

    currentDiv = stock.dividends[stock.dividends.index.year == currentYear][0]
    currentPrice = stock.history(period="1d")["Close"][0]
    divYield = currentDiv / currentPrice
    print(f"The dividendYield of {stockTicker} at year {currentYear} is: {divYield}")

In [26]:
getDividendYield("NXI.PA", 2020)

The dividendYield of NXI.PA at year 2020 is: 0.14619882714792826


In [39]:
def getDivPerSharePeriodMean(stockTicker:str, period:int=5)->float:
    """
    This function returns the mean of the dividend per share over a given period. By default, the period is 5 years.
    :param stockTicker:
    :type stockTicker:
    :param period:
    :type period:
    :return:
    :rtype:
    """
    currentYear=datetime.datetime.now().year
    stock = yf.Ticker(stockTicker)
    total = 0
    for i in range(period):
        try:
            currentDiv = stock.dividends[stock.dividends.index.year == currentYear][0]
        except IndexError:
            currentDiv = 0
        print(f"The dividend of {stockTicker} at year {currentYear} is: {currentDiv}")
        total +=currentDiv
        currentYear -=1
    print(total)
    if total>0:
        divMean = total/period
    else:
        divMean = 0
    print(divMean)
    return divMean

In [40]:
getDivPerSharePeriodMean("NXI.PA")

The dividend of NXI.PA at year 2023 is: 2.5
The dividend of NXI.PA at year 2022 is: 2.5
The dividend of NXI.PA at year 2021 is: 2.0
The dividend of NXI.PA at year 2020 is: 2.0
The dividend of NXI.PA at year 2019 is: 2.5
11.5
2.3


2.3

In [41]:
divMeanList = []
for ticker in tickerList:
    print(ticker)
    divMean = {"symbol": ticker,"divPerShare5YearMean": getDivPerSharePeriodMean(ticker)}
    divMeanList.append(divMean)

divMeanDf= pd.DataFrame(divMeanList)
divMeanDf.set_index("symbol",inplace=True)

divMeanDf.head(5)

AF.PA
The dividend of AF.PA at year 2023 is: 0
The dividend of AF.PA at year 2022 is: 0
The dividend of AF.PA at year 2021 is: 0
The dividend of AF.PA at year 2020 is: 0
The dividend of AF.PA at year 2019 is: 0
0
0
VIRP.PA
The dividend of VIRP.PA at year 2023 is: 1.32
The dividend of VIRP.PA at year 2022 is: 1.25
The dividend of VIRP.PA at year 2021 is: 0.75
The dividend of VIRP.PA at year 2020 is: 0
The dividend of VIRP.PA at year 2019 is: 0
3.3200000000000003
0.664
VCT.PA
The dividend of VCT.PA at year 2023 is: 1.65
The dividend of VCT.PA at year 2022 is: 1.65
The dividend of VCT.PA at year 2021 is: 1.5
The dividend of VCT.PA at year 2020 is: 1.5
The dividend of VCT.PA at year 2019 is: 1.5
7.8
1.56
LAT.PA
The dividend of LAT.PA at year 2023 is: 0
The dividend of LAT.PA at year 2022 is: 0
The dividend of LAT.PA at year 2021 is: 0
The dividend of LAT.PA at year 2020 is: 0
The dividend of LAT.PA at year 2019 is: 0
0
0
GBT.PA
The dividend of GBT.PA at year 2023 is: 0.5
The dividend of GB

,divPerShare5YearMean
symbol,
AF.PA,0.000
VIRP.PA,0.664
VCT.PA,1.560
LAT.PA,0.000
GBT.PA,0.720


In [44]:
stockDivSummary = pd.merge(sortByDiv,divMeanDf,on="symbol",how="inner")
stockDivSummary.head(20)

,shortName,longBusinessSummary,dividendYield,marketCap,beta,forwardPE,currentPrice,targetHighPrice,targetLowPrice,divPerShare5YearMean
symbol,,,,,,,,,,
NXI.PA,NEXITY,Nexity SA operates as a real estate company in...,0.1809,7.596621e+08,1.236563,6.408450,13.650,28.9,15.0,2.300000
ICAD.PA,ICADE,As a commercial property investor (portfolio w...,0.1310,2.495086e+09,1.179031,7.377575,32.240,40.0,40.0,2.196000
ALD.PA,ALD,ALD S.A. provides service leasing and vehicle ...,0.1215,7.408224e+09,0.953822,5.213018,8.810,20.1,15.0,0.744802
ALTA.PA,ALTAREA,Altarea is the French leader in low-carbon urb...,0.1199,1.728011e+09,1.083270,14.850087,84.200,103.0,101.0,9.891554
MERY.PA,MERCIALYS,Mercialys is one of France's leading real esta...,0.1191,7.414276e+08,1.571158,8.777778,7.900,12.0,7.5,0.682000
COFA.PA,COFACE,"COFACE SA, through its subsidiaries, provides ...",0.1177,1.930410e+09,0.880393,8.736486,12.930,17.2,13.0,0.872000
ENGI.PA,ENGIE,"ENGIE SA engages in the power, natural gas, an...",0.0965,3.603044e+10,0.997728,8.653488,14.884,21.2,14.2,0.790000
KOF.PA,KAUFMAN ET BROAD,Kaufman & Broad S.A. operates as a property de...,0.0958,5.229205e+08,1.055261,9.132841,24.750,35.0,24.5,2.090000
COV.PA,COVIVIO,"Thanks to its partnering history, its real est...",0.0925,4.003320e+09,1.460635,9.638554,40.000,62.0,44.0,4.100000
